In [8]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# data I/O
data = open('/content/drive/My Drive/Colab Notebooks/KSA_EDU/Module_06/input_flower.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has {} characters, {} unique'.format(data_size, vocab_size))
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

data has 86 characters, 34 unique


In [0]:
# hyperparameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [0]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(vocab_size, hidden_size)*0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [0]:
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) +by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0])
    # backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [0]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = [ ]
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [49]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length
while True:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n {} \n----'.format(txt, ))
        
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter {}, loss : {}'.format(n, smooth_loss))
        
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                 [dWxh, dWhh, dWhy, dbh, dby],
                                 [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
        
    p += seq_length
    n += 1

----
 하었었 았었기지하나름하가짓않나그았러짓 었내서하서름 로기로다름지전게었의않와서을의되짓다가는에게전에름었는
만짓게꽃그로게러몸되지지와름었다와았짓몸다름그 몸는때았
름내.서서러나꽃로 않주름았와와만그만않로다지게을름 었
에내주나나와름나게로와름내러기에았았로을에주.을다
 이의에다않주름름.그꽃되전로는가전의전었내을만몸서에하내가내는않때는
그러몸서에
나짓이가의
기주의에는내 나.그서주 
----
iter 0, loss : 88.1590227773135
----
 기내불그는만전이이는 와기을 의주불의  다불름게
전게이
에지름는는 에의그다주다주에의 을게주에주와다그불이이 짓그불만 때 만 는 지그그을름나주는에게주주는는불주이이 이 러 불만만는주전주러그는 와을의
불의  러그는이게만하내에와
 름이그러주는을불이그의그는그불불불는 러주때을는에러 때을는러 는을을러을만 서에러기전몸전불 
는그불전게.게이 러와전 러에지
의의서 는 는 의 내 
----
iter 100, loss : 88.87550743752176
----
  지았지 때 지 의았와 에을 의 불지와
 지 . 나러을그의나다지 다름 이나않때 때 을 러 다의 하.와가
 . 그었름몸름다을 의짓않 는 
 하았에 지나러하않하 다는 는 때하
..을.때지않내기 는 지 
았이의 내 나전 불는 

다지 의 전

이의 의 다름몸

주는 나 다짓을 
서다
 는
 
주에만 가에나다서 
 나 의 을 는 에그나을 의 게의나
 몸가내을 않짓 
----
iter 200, loss : 86.2673262566613
----
 가 그는 
하전의을불그았게로 나에 에는 서 지 
나전 이름을 지만이가 그의 때 와만 다만 
그는 는 와.게로 와. 
러주로 그가 에을 때 지나로 이짓에게만 않 그나에을에나 로 않 을 
그는 는 않에는 다만 않았나지 몸.지나에 .서았게서 전와지하는짓
 때 불러 나내 
 에서 나에이게나. 때 불지나가 주의 지 
 불
주의 전에는 
그는 몸짓에전와서 지 않그었  
----
iter 300, loss : 81.80668659485696
----
 와러 었을 

KeyboardInterrupt: ignored